# Phase 2 — Cleaning & Feature Engineering

**Goal:** Turn raw/interim tables into a clean, feature-ready dataset for modeling.

**Inputs:** Phase-1 outputs (video + channel tables), utility helpers.

**Outputs (example):**
- `data/processed/2025-09-18/ml_table.parquet`  (+ CSV if enabled)

**Assumptions:**
- Timestamps are handled in **UTC**.
- Text fields may be empty; features are built to handle missing values.


In [41]:
# ===== 1) SETUP =====
# --- Standard library ---
import os
import sys
import re
import json
import math
import ast
import textwrap
import inspect
import hashlib
import platform
from pathlib import Path
from collections import defaultdict, Counter
from datetime import datetime, timezone

# --- Third-party ---
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [ ]:

# ===== 2) CONFIG  =====
# How Viral is defined: computed an engagement score per video (likes/comments) and then mark the
# TOP x% within a *cohort* as positive (viral=1). Everything else is 0.
TOP_FRAC   = 0.10      # top 10% of the cohort receive viral=1         

COHORT     = "category_day"     # "category_day" (category × upload date) or "category"

# Engagement weighting:
# Formed a scalar score used for RANKING (not absolute measurement).
# score = W_LIKE * likes + W_COMM * comments   (computed within each cohort)
W_LIKE     = 0.5                # engagement weight for likes
W_COMM     = 0.5                # engagement weight for comments

SAVE_CSV   = True              # also write CSV next to the Parquet


In [27]:
# ===== 3) Locating and Loading Dataset  =====
def find_repo_root(start: Path = Path.cwd(), must_have=("data",)):
    """
    Walk upward from `start` until we find a directory that contains ALL entries in `must_have`.
    Used to robustly locate the repo root (expected to contain 'data/').
    """
    p = start
    while True:
        if all((p / m).exists() for m in must_have):
            return p
        if p.parent == p:  # hit filesystem root without success
            raise FileNotFoundError(f"Could not find a repo root above {start} containing: {must_have}")
        p = p.parent

# 1) Locate repo root (has a 'data/' folder)
REPO_ROOT = find_repo_root(Path.cwd(), must_have=("data",))
print("Repo root:", REPO_ROOT)

# 2) (Recommended) Switch CWD to repo root so relative paths resolve as 'repo/...'
os.chdir(REPO_ROOT)
print("CWD set to:", Path.cwd())

# 3) Attempt to auto-pick a Phase-2 table from common locations under data/processed/
SEARCH_DIRS = [
    REPO_ROOT / "data" / "processed",
    REPO_ROOT / "data" / "processed" / "phase2_prototype",
]

# If preference or an explicit INPUT_PATH earlier
PREFERRED_FILE = globals().get("PREFERRED_FILE", None)
INPUT_PATH = globals().get("INPUT_PATH", None)

# Prefer an explicit filename in data/processed/ if given.
if PREFERRED_FILE:
    candidate = REPO_ROOT / "data" / "processed" / PREFERRED_FILE
    if candidate.exists():
        INPUT_PATH = str(candidate.resolve())

# if not pick the newest file that matches the patterns
if not INPUT_PATH:
    hit = latest_match(PATTERNS, SEARCH_DIRS)
    if hit:
        INPUT_PATH = str(hit.resolve())

print("Using INPUT_PATH:", INPUT_PATH)


Repo root: /Users/asmitabisht/Desktop/repo
CWD set to: /Users/asmitabisht/Desktop/repo
Using INPUT_PATH: data/processed/all_days.csv


In [44]:

if p.suffix.lower() == ".parquet":
    df = pd.read_parquet(p)
elif p.suffix.lower() == ".csv":
    df = pd.read_csv(p)

print("Rows:", len(df), "Cols:", len(df.columns))
print("Sample columns:", list(df.columns[:25]))


Rows: 5240 Cols: 20
Sample columns: ['approx_age_hours', 'categoryId', 'channelId', 'channel_hiddenSubscriberCount', 'channel_subscriberCount', 'channel_videoCount', 'channel_viewCount', 'comments_72h', 'description', 'duration_sec', 'has_missing_stats', 'is_shorts', 'likes_72h', 'publishedAt_utc', 'snapshot_at_utc', 'source_file', 'tags_str', 'title', 'videoId', 'views_72h']


In [29]:

# ===== 4) HELPERS  =====

# ISO8601 duration parser (PT#H#M#S, P#DT#H#M#S)
_ISO_DUR_RE = re.compile(
    r'^P(?:(?P<days>\d+)D)?(?:T(?:(?P<hours>\d+)H)?(?:(?P<minutes>\d+)M)?(?:(?P<seconds>\d+)S)?)?$'
)

def parse_iso8601_duration_to_seconds(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float)) and not math.isnan(x):
        return float(x)
    s = str(x).strip()
    m = _ISO_DUR_RE.match(s)
    if not m:
        return np.nan
    d = int(m.group('days') or 0)
    h = int(m.group('hours') or 0)
    mnt = int(m.group('minutes') or 0)
    sec = int(m.group('seconds') or 0)
    return float(d*86400 + h*3600 + mnt*60 + sec)

def text_is_missing(x: object) -> int:
    #1 if title/description is NaN/empty/whitespace, else 0 
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return 1
    return 0 if str(x).strip() else 1

# returns the number of whitespaced separated tokens in text
def safe_len_words(text):
    if pd.isna(text): return 0
    return len(str(text).split())

# return character length of text after coercion to str
def safe_len_chars(text):
    if pd.isna(text): return 0
    return len(str(text))

# return number of upercase letters
def caps_ratio(text):
    if pd.isna(text): return 0.0
    s = re.sub(r'[^A-Za-z]', '', str(text))
    if not s: return 0.0
    upp = sum(c.isupper() for c in s)
    return upp / len(s)

#counts occurances of a specific character in text 
def count_char(text, ch):
    if pd.isna(text): return 0
    return str(text).count(ch)

# calculate tags count
def tags_count(x):
    if pd.isna(x): return 0
    if isinstance(x, (list, tuple)): return len(x)
    s = str(x)
    # Try JSON-like list
    if s.startswith('[') and s.endswith(']'):
        try:
            import ast
            lst = ast.literal_eval(s)
            return len(lst) if isinstance(lst, list) else 0
        except Exception:
            pass
    # comma separated
    if ',' in s: return len([t for t in s.split(',') if t.strip()])
    return 1 if s.strip() else 0

# Turn tags into a space-separated 'bag of tags' string. Missing -> '' (empty string), which is safe for TF-IDF.
def tags_to_text(x: object) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    toks = []
    if isinstance(x, (list, tuple)):
        toks = [str(t).strip().lower() for t in x if str(t).strip()]
    else:
        s = str(x)
        if s.startswith('[') and s.endswith(']'):
            try:
                lst = ast.literal_eval(s)
                toks = [str(t).strip().lower() for t in lst if str(t).strip()]
            except Exception:
                toks = [t.strip().lower() for t in re.split(r'[|,]', s) if t.strip()]
        else:
            toks = [t.strip().lower() for t in re.split(r'[|,]', s) if t.strip()]
    # keep multi-word tags as a single token
    toks = [t.replace(' ', '_') for t in toks]
    return " ".join(toks)

def to_utc_ts(x):
    # parses ISO timestamps, returns timezone-aware pandas Timestamp or NaT
    return pd.to_datetime(x, utc=True, errors='coerce')


In [ ]:

# ===== 5) FEATURE ENGINEERING  =====

df_feat = df.copy()

# ensure expected non-tag columns exist
for col in [
    'title','description','categoryId','publishedAt_utc','duration','duration_sec',
    'channel_subscriberCount','channel_viewCount','channel_videoCount',
    'likes_72h','comments_72h'
]:
    if col not in df_feat.columns:
        df_feat[col] = np.nan

# unify raw tags into df_feat['tags'] ===
if 'tags' not in df_feat.columns:
    if 'tags_str' in df_feat.columns:
        df_feat['tags'] = df_feat['tags_str']          # comma/whitespace-separated
    elif 'tags_json' in df_feat.columns:
        df_feat['tags'] = df_feat['tags_json']         # JSON list as string
    else:
        df_feat['tags'] = np.nan                       # truly missing


# publish time buckets
ts = df_feat['publishedAt_utc'].apply(to_utc_ts)
df_feat['pub_hour']   = ts.dt.hour.astype('float')
df_feat['pub_dow']    = ts.dt.weekday.astype('float')
df_feat['upload_day'] = ts.dt.date.astype('string')  # for category × day cohorting

# cyclical time features (works better than raw hour)
import numpy as np
df_feat['pub_hour_sin'] = np.sin(2*np.pi*df_feat['pub_hour']/24)
df_feat['pub_hour_cos'] = np.cos(2*np.pi*df_feat['pub_hour']/24)

# duration (seconds) — if missing, parse ISO 8601 string
if 'duration_sec' not in df_feat.columns or df_feat['duration_sec'].isna().all():
    df_feat['duration_sec'] = df_feat['duration'].apply(parse_iso8601_duration_to_seconds)

# text features (title/description); intentionally simple fast heuristics
df_feat['title_len_words']    = df_feat['title'].apply(safe_len_words)
df_feat['title_len_chars']    = df_feat['title'].apply(safe_len_chars)
df_feat['title_caps_ratio']   = df_feat['title'].apply(caps_ratio)
df_feat['title_num_exclaim']  = df_feat['title'].apply(lambda s: count_char(s,'!'))
df_feat['title_num_question'] = df_feat['title'].apply(lambda s: count_char(s,'?'))
df_feat['desc_len_words']     = df_feat['description'].apply(safe_len_words)

# missingness indicators
df_feat['title_missing']       = df_feat['title'].apply(text_is_missing).astype(int)
df_feat['description_missing'] = df_feat['description'].apply(text_is_missing).astype(int)

# binary features and emoji counts
df_feat['title_has_question'] = df_feat['title'].str.contains(r'\?').fillna(False).astype(int)
df_feat['title_has_exclaim']  = df_feat['title'].str.contains(r'!').fillna(False).astype(int)
df_feat['title_emoji_count']  = df_feat['title'].str.count(r'[\U00010000-\U0010ffff]').fillna(0)
df_feat['desc_emoji_count']   = df_feat['description'].str.count(r'[\U00010000-\U0010ffff]').fillna(0)

# tags
df_feat['tags_count'] = df_feat['tags'].apply(tags_count)
df_feat['has_tags']   = (df_feat['tags_count'] > 0).astype(int)
df_feat['tags_text']  = df_feat['tags'].apply(tags_to_text)


# channel scale features 
for c in ['channel_subscriberCount','channel_viewCount','channel_videoCount']:
    lc = c + '_log1p'
    df_feat[lc] = np.log1p(pd.to_numeric(df_feat[c], errors='coerce'))

# duration log
df_feat['duration_log1p'] = np.log1p(pd.to_numeric(df_feat['duration_sec'], errors='coerce'))

# engagement score 
likes = pd.to_numeric(df_feat['likes_72h'], errors='coerce').fillna(0)
comms = pd.to_numeric(df_feat['comments_72h'], errors='coerce').fillna(0)
df_feat['engagement_score'] = W_LIKE*np.log1p(likes) + W_COMM*np.log1p(comms)

print("Engineered feature columns now present:")
print([c for c in df_feat.columns
       if c.endswith('_log1p')
       or c.startswith(('title_','desc_','tags_count','pub_','has_tags','engagement_score'))])

Engineered feature columns now present:
['pub_hour', 'pub_dow', 'pub_hour_sin', 'pub_hour_cos', 'title_len_words', 'title_len_chars', 'title_caps_ratio', 'title_num_exclaim', 'title_num_question', 'desc_len_words', 'title_missing', 'title_has_question', 'title_has_exclaim', 'title_emoji_count', 'desc_emoji_count', 'tags_count', 'has_tags', 'channel_subscriberCount_log1p', 'channel_viewCount_log1p', 'channel_videoCount_log1p', 'duration_log1p', 'engagement_score']


In [ ]:
# ===== 6) COHORT PERCENTILES & LABELS  =====
# cohort for percentile: category × upload_day (or just category)
if COHORT == "category_day" and 'upload_day' in df_feat.columns:
    grp = df_feat.groupby(['categoryId','upload_day'], dropna=False)['engagement_score']
else:
    grp = df_feat.groupby(['categoryId'], dropna=False)['engagement_score']

# percentile within cohort and top-frac label
df_feat['engagement_percentile'] = grp.rank(pct=True, method='average')
df_feat['viral'] = (df_feat['engagement_percentile'] >= (1.0 - TOP_FRAC)).astype(int)

print("Label prevalence (overall):", df_feat['viral'].mean().round(3))
try:
    print("Per-category sample prevalence:")
    print(df_feat.groupby('categoryId')['viral'].mean().round(3).head(10))
except Exception:
    pass


Label prevalence (overall): 0.132
Per-category sample prevalence:
categoryId
1     0.171
2     0.308
10    0.167
15    0.571
17    0.115
19    0.536
20    0.110
22    0.118
23    0.179
24    0.111
Name: viral, dtype: float64


In [38]:

# ===== 7) SELECT OUTPUT COLUMNS  =====

# selecting what is kept: core IDs + features + raw text + engagement fields + label
keep = [
  # ids/meta
    'videoId','channelId','categoryId','publishedAt_utc','upload_day',
    # numeric features
    'duration_log1p','tags_count','title_len_words','title_len_chars','title_caps_ratio',
    'title_num_exclaim','title_num_question','desc_len_words','pub_hour','pub_dow',
    'channel_subscriberCount_log1p','channel_viewCount_log1p','channel_videoCount_log1p', 'title_has_question','title_has_exclaim',
    'title_emoji_count','desc_emoji_count',
    'pub_hour_sin','pub_hour_cos',
    # binary “presence/missing”
    'has_tags','title_missing','description_missing',
    # text passthrough
    'title','description','tags_text',
    # engagement + label
    'likes_72h','comments_72h','engagement_score','engagement_percentile','viral'
]

exists = [c for c in keep if c in df_feat.columns]
df_out = df_feat[exists].copy()

print("Columns in output:", len(exists))
print(exists)
print("Rows:", len(df_out), "Viral rate:", df_out['viral'].mean().round(3))


Columns in output: 35
['videoId', 'channelId', 'categoryId', 'publishedAt_utc', 'upload_day', 'duration_log1p', 'tags_count', 'title_len_words', 'title_len_chars', 'title_caps_ratio', 'title_num_exclaim', 'title_num_question', 'desc_len_words', 'pub_hour', 'pub_dow', 'channel_subscriberCount_log1p', 'channel_viewCount_log1p', 'channel_videoCount_log1p', 'title_has_question', 'title_has_exclaim', 'title_emoji_count', 'desc_emoji_count', 'pub_hour_sin', 'pub_hour_cos', 'has_tags', 'title_missing', 'description_missing', 'title', 'description', 'tags_text', 'likes_72h', 'comments_72h', 'engagement_score', 'engagement_percentile', 'viral']
Rows: 5240 Viral rate: 0.132


In [ ]:
# ===== 8) SAVE OUTPUTS  =====

# ---- Config 
SAVE_PARQUET = True
SAVE_CSV     = True                 # write CSV alongside Parquet
CSV_ENCODING = "utf-8"              # use 'utf-8-sig' if opening with old Excel (for testing)
CSV_INDEX    = False
PARQUET_COMPRESSION = "snappy"    

# ---- find repo root so its always saved to repo/data/processed
def _find_repo_root(start: Path = Path.cwd(), must_have=("data",)) -> Path:
    p = start
    while True:
        if all((p / m).exists() for m in must_have):
            return p
        if p.parent == p:
            # Fallback: create data/ in the current dir if not found
            (start / "data").mkdir(parents=True, exist_ok=True)
            return start
        p = p.parent

REPO_ROOT = _find_repo_root()
PROCESSED_DIR = REPO_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

#  optional for google collab testing
print("Repo root:", REPO_ROOT.as_posix())
print("Saving under:", PROCESSED_DIR.as_posix())

# ---- filenames
stamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")
base_name = f"ml_table_{stamp}"
out_pq  = PROCESSED_DIR / f"{base_name}.parquet"
out_csv = PROCESSED_DIR / f"{base_name}.csv"
sum_js  = PROCESSED_DIR / f"{base_name}_summary.json"

# ---- write parquet / CSV
if SAVE_PARQUET:
    df_out.to_parquet(out_pq, index=False, compression=PARQUET_COMPRESSION)

if SAVE_CSV:
    df_out.to_csv(out_csv, index=CSV_INDEX, encoding=CSV_ENCODING)

# ---- summary JSON 
summary = {
    "generated_utc": stamp,
    "input_path": str(globals().get("INPUT_PATH", "")),
    "rows": int(len(df_out)),
    "viral_rate": float(df_out["viral"].mean()) if "viral" in df_out.columns else None,
    "cohort": globals().get("COHORT", None),
    "top_frac": globals().get("TOP_FRAC", None),
    "weights": {
        "likes":  globals().get("W_LIKE", None),
        "comments": globals().get("W_COMM", None)
    },
    "columns": list(df_out.columns),
    "output_parquet": out_pq.as_posix() if SAVE_PARQUET else None,
    "output_csv": out_csv.as_posix() if SAVE_CSV else None
}
with open(sum_js, "w") as f:
    json.dump(summary, f, indent=2)

# ---- console prints
print("Saved:", out_pq.as_posix() if SAVE_PARQUET else "(Parquet disabled)")
print("Saved:", out_csv.as_posix() if SAVE_CSV else "(CSV disabled)")
print("Saved:", sum_js.as_posix())
print(json.dumps(summary, indent=2))

# ---- notebook links (nice to click)
links = []
if SAVE_PARQUET: links.append(f'<li><a href="{out_pq.as_posix()}">{out_pq.name}</a></li>')
if SAVE_CSV:     links.append(f'<li><a href="{out_csv.as_posix()}">{out_csv.name}</a></li>')
links.append(f'<li><a href="{sum_js.as_posix()}">{sum_js.name}</a></li>')
display(HTML(f"""
<div style="margin:8px 0;padding:10px 12px;border-left:4px solid #4a90e2;background:#f7f9fc">
  <b>Artifacts saved to:</b> <code>{PROCESSED_DIR.as_posix()}</code>
  <ul style="margin:6px 0 0 18px">{''.join(links)}</ul>
</div>
"""))

# ---- Optional: auto-download in Colab
if "google.colab" in sys.modules:
    try:
        from google.colab import files
        if SAVE_CSV and out_csv.exists():
            files.download(out_csv.as_posix())
        elif SAVE_PARQUET and out_pq.exists():
            files.download(out_pq.as_posix())
    except Exception as e:
        print("Colab download skipped:", e)


Repo root: /Users/asmitabisht/Desktop/repo
Saving under: /Users/asmitabisht/Desktop/repo/data/processed
Saved: /Users/asmitabisht/Desktop/repo/data/processed/ml_table_2025-09-22T03-12-02Z.parquet
Saved: /Users/asmitabisht/Desktop/repo/data/processed/ml_table_2025-09-22T03-12-02Z.csv
Saved: /Users/asmitabisht/Desktop/repo/data/processed/ml_table_2025-09-22T03-12-02Z_summary.json
{
  "generated_utc": "2025-09-22T03-12-02Z",
  "input_path": "data/processed/all_days.csv",
  "rows": 5240,
  "viral_rate": 0.1316793893129771,
  "cohort": "category_day",
  "top_frac": 0.1,
  "weights": {
    "likes": 0.5,
    "comments": 0.5
  },
  "columns": [
    "videoId",
    "channelId",
    "categoryId",
    "publishedAt_utc",
    "upload_day",
    "duration_log1p",
    "tags_count",
    "title_len_words",
    "title_len_chars",
    "title_caps_ratio",
    "title_num_exclaim",
    "title_num_question",
    "desc_len_words",
    "pub_hour",
    "pub_dow",
    "channel_subscriberCount_log1p",
    "channel_